In [58]:
import pandas as pd # Pour le dataframe
from skimage.transform import resize
import numpy as np # Pour la normalisation et calculs de moyenne
import matplotlib.pyplot as plt # Pour la visualisation

from PIL import Image

import librosa # Pour l'extraction des features et la lecture des fichiers wav
import librosa.display # Pour récupérer les spectrogrammes des audio
import librosa.feature

import os # C'est ce qui va nous permettre d'itérer sur les fichiers de l'environnement de travail

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve, RandomizedSearchCV # Split de dataset et optimisation des hyperparamètres
from sklearn.ensemble import RandomForestClassifier # Random forest
from sklearn.ensemble import GradientBoostingClassifier # XGBoost
from sklearn.neighbors import KNeighborsClassifier # k-NN
from sklearn.svm import SVC # SVM
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score, zero_one_loss, classification_report # Métriques pour la mesure de performances
from sklearn.preprocessing import normalize, StandardScaler

import tensorflow as tf # Pour le reseau de neurones simple et pour le CNN
from tensorflow.keras.models import load_model

import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from keras.optimizers import Adam
from keras.utils import to_categorical

from xgboost import XGBClassifier
from pprint import pprint

### Chargement du CSV

In [59]:
df = pd.read_csv('./Data/1Mdata.csv')

In [60]:
df.head()

,analysis_sample_rate,duration,track_id,danceability,loudness,tempo,start_of_fade_out,end_of_fade_in,song_id,key,key_confidence,time_signature,genres
0,22050,156.55138,TRMMMKD128F425225D,0.0,-10.555,150.778,148.660,0.258,SOVFVAK12A8C1350D9,9,0.808,1,Pop
1,22050,138.97098,TRMMMRX128F93187D9,0.0,-2.060,177.768,138.971,0.000,SOGTUKN12AB017F4F1,7,0.418,4,Electronica/Dance
2,22050,212.37506,TRMMMLR128F1494097,0.0,-4.931,101.450,206.629,0.000,SOQVRHI12A6D4FB2D7,3,0.611,1,Reggae
3,22050,221.20444,TRMMMBB12903CB7D21,0.0,-12.214,98.020,212.120,0.165,SOEYRFT12AB018936C,11,0.332,4,Hip-Hop
4,22050,104.48934,TRMMMML128F4280EE9,0.0,-20.160,124.339,104.489,0.000,SOJCFMH12A8C13B0C2,4,0.668,4,Cadence


##### Afficher le total de titres par genres :

In [61]:
genre_titles_counts = df['genres'].value_counts()
print(genre_titles_counts)

genres
Rock                  261242
Pop                    57210
Electronic             38235
Jazz                   37844
Hip-Hop                31580
                       ...  
house speed garage         1
Novelas                    1
House/fonky                1
Cast Recording             1
Pop/80's/New Wave          1
Name: count, Length: 6152, dtype: int64


##### Compter les genres uniques :

In [62]:
unique_genres_count = df['genres'].nunique()
print(f"Number of unique genres: {unique_genres_count}")

Number of unique genres: 6152


##### Identification des genres minoritaires :
    Identifiez les genres qui ont un nombre très faible de titres. Vous pouvez fixer un seuil en dessous duquel les genres seront considérés comme trop minoritaires.

In [63]:
threshold = 500  # Seuil
genre_counts = df['genres'].value_counts()
minority_genres = genre_counts[genre_counts <= threshold].index
print(minority_genres)

Index(['Reggaeton', 'zouk', 'Tango', 'Bhangra', 'Doo Wop', 'Freestyle',
       'Southern Gospel', 'Vallenato', 'Soca', 'Oi!',
       ...
       'Irish Jive', 'Hands', 'Vocal Classical', 'Funk/Punk', 'Male Vocal',
       'house speed garage', 'Novelas', 'House/fonky', 'Cast Recording',
       'Pop/80's/New Wave'],
      dtype='object', name='genres', length=6113)


##### Quelles sont les implications concernant ce résultat ?
**1. Distribution très déséquilibrée :**
Notre ensemble de données est très déséquilibré, avec quelques genres très bien représentés et une grande majorité de genres avec peu de titres.

**2. Risque de surapprentissage :**
Avec une telle distribution, il y a un risque élevé que notre modèle soit biaisé en faveur des genres bien représentés.

In [64]:
threshold = 2000  # Seuil
genre_counts = df['genres'].value_counts()
to_remove = genre_counts[genre_counts <= threshold].index
df = df[~df['genres'].isin(to_remove)]

In [65]:
unique_genres_count = df['genres'].nunique()
print(f"Number of unique genres after operation: {unique_genres_count}")

Number of unique genres after operation: 26


##### Résultats après le tri
**1. Réduction significative :**
Nous avons considérablement réduit le nombre de genres, ce qui devrait faciliter l'entraînement du modèle et améliorer sa généralisation.

**2. Focus sur les genres populaires :**
Avec seulement 26 genres restants, votre modèle sera très axé sur les genres les plus populaires ou les plus couramment représentés dans votre ensemble de données.

**3. Risque de perte d'information :**
Il est important de noter que cette approche pourrait entraîner la perte d'informations sur les genres moins courants, ce qui pourrait être pertinent selon l'objectif de notre projet.

___

##### Affichage des genres restants, ainsi que du nombre de titres par genres

In [66]:
remaining_genres = df['genres'].unique()
print("Remaining genres after applying the threshold:")
print(remaining_genres)

Remaining genres after applying the threshold:
['Pop' 'Electronica/Dance' 'Reggae' 'Hip-Hop' 'Rock' 'World'
 'Gospel & Religious' 'Folk' 'Latin' 'Other' 'Techno' 'R&B' 'Country'
 'Jazz' 'Hardcore' 'House' 'Classical' 'Soundtrack' 'Easy Listening'
 'Trance' 'New Age' 'Electronic' 'Dance' 'Drum & Bass' 'Christmas'
 'Comedy']


In [67]:
remaining_genre_counts = df['genres'].value_counts()
print("Remaining genres and their counts after applying the threshold:")
print(remaining_genre_counts)

Remaining genres and their counts after applying the threshold:
genres
Rock                  261242
Pop                    57210
Electronic             38235
Jazz                   37844
Hip-Hop                31580
R&B                    25712
Country                23318
Latin                  21610
Reggae                 20139
Folk                   12994
Soundtrack             12151
Dance                  10887
Electronica/Dance      10713
World                  10634
Gospel & Religious      7937
House                   6357
Trance                  5643
New Age                 4880
Other                   4590
Techno                  4533
Classical               3837
Easy Listening          3832
Christmas               3260
Hardcore                3171
Comedy                  2984
Drum & Bass             2224
Name: count, dtype: int64


___

##### Equilibrage des classes :
**Sous-échantillonnage :**
Nous pouvons réduire le nombre d'échantillons des classes sur-représentées pour les équilibrer avec les classes moins représentées.

In [68]:
# Nombre d'échantillons à conserver pour la classe "Rock"
n_samples_rock = 60000  # Vous pouvez ajuster ce nombre selon vos besoins

# Séparer les données de la classe "Rock" et des autres classes
df_rock = df[df['genres'] == 'Rock']
df_other = df[df['genres'] != 'Rock']

# Sous-échantillonnage de la classe "Rock"
df_rock_sampled = df_rock.sample(n=n_samples_rock, random_state=1)

# Fusionner les données sous-échantillonnées de "Rock" avec les autres données
df_balanced = pd.concat([df_rock_sampled, df_other])

# Vérifier la nouvelle distribution des classes
new_genre_counts = df_balanced['genres'].value_counts()
print("New genre distribution:")
print(new_genre_counts)

New genre distribution:
genres
Rock                  60000
Pop                   57210
Electronic            38235
Jazz                  37844
Hip-Hop               31580
R&B                   25712
Country               23318
Latin                 21610
Reggae                20139
Folk                  12994
Soundtrack            12151
Dance                 10887
Electronica/Dance     10713
World                 10634
Gospel & Religious     7937
House                  6357
Trance                 5643
New Age                4880
Other                  4590
Techno                 4533
Classical              3837
Easy Listening         3832
Christmas              3260
Hardcore               3171
Comedy                 2984
Drum & Bass            2224
Name: count, dtype: int64


**Supprimer "Soundtrack" :**
Étant donné que "Soundtrack" peut être un mélange de différents genres et styles, il peut être difficile pour le modèle de le classer de manière significative. Le supprimer pourrait simplifier le problème de classification.

**Fusionner "Other" et "Easy Listening" :**
Si "Other" est une catégorie fourre-tout et que "Easy Listening" est un genre moins représenté, les fusionner peut aider à équilibrer les classes tout en réduisant le nombre de catégories à prédire.

In [69]:
# Supprimer les lignes où le genre est "Soundtrack"
df_balanced = df_balanced[df_balanced['genres'] != 'Soundtrack']

# Fusionner les genres "Other" et "Easy Listening" en une seule catégorie "Other/Easy Listening"
df_balanced['genres'] = df_balanced['genres'].apply(lambda x: 'Other/Easy Listening' if x in ['Other', 'Easy Listening'] else x)

# Vérifier la nouvelle distribution des genres
new_genre_counts = df_balanced['genres'].value_counts()
print("New genre distribution after adjustments:")
print(new_genre_counts)


New genre distribution after adjustments:
genres
Rock                    60000
Pop                     57210
Electronic              38235
Jazz                    37844
Hip-Hop                 31580
R&B                     25712
Country                 23318
Latin                   21610
Reggae                  20139
Folk                    12994
Dance                   10887
Electronica/Dance       10713
World                   10634
Other/Easy Listening     8422
Gospel & Religious       7937
House                    6357
Trance                   5643
New Age                  4880
Techno                   4533
Classical                3837
Christmas                3260
Hardcore                 3171
Comedy                   2984
Drum & Bass              2224
Name: count, dtype: int64


___

Mainteanant que notre CSV à l'air relativmenet exploitable, nous allons le sauvegarder et xréer quelques échantillons audio

In [73]:
# Enregistrer le DataFrame dans un nouveau fichier CSV
df_balanced.to_csv('balanced_genre_dataset.csv', index=False)

In [74]:
df = pd.read_csv('./Data/balanced_genre_dataset.csv')